# Otimização de parâmetros

In [1]:
from ztools.ntf import telegram # apagar - biblioteca particular

from agent import Agent
from environment import Environment
from utils import plot_evolution

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow.compat.v1 as tf

from tqdm import tqdm

In [2]:
def plot_evolution(rewards: list, pack_size: int, game_name: str):
    to_plot = []
    mean = 0
    for i, reward in enumerate(rewards):
        if i % pack_size == 0:
            to_plot.append(mean / pack_size)
            mean = 0
        mean += reward
    plt.figure(figsize=(15, 4))
    plt.plot(range(len(to_plot)), to_plot)
    plt.axis((0, len(to_plot), -25, 50))
    plt.xlabel(f"Pack de {pack_size} episódios")
    plt.ylabel("Score")
    plt.title(f"{game_name} DRL para lançamento oblíquo")
    plt.show()


In [ ]:
class Environment:
    # inicia parametros do objeto
    def __init__(self, theta_disc: int, vel_disc: int, max_dist: float, target_len: float):

        max_theta = 90  # ângulo máximo de lançamento
        d_theta = max_theta / theta_disc  # discretização dos possíveis ângulos

        max_vel = np.sqrt(max_dist * 9.81 / 2)  # máxima velocidade de lançamento possível
        d_vel = max_vel / vel_disc  # discretização das possíveis velocidades

        self.success_reward = 50  # recompensa para um acerto

        self.max_dist = max_dist  # máxima distância do alvo
        self.target_len = target_len  # comprimento do alvo (tolerância absoluta)
        self.random_target_len = type(target_len) not in [float, int]  # definição de comprimento aleatório do alvo
        self.theta_range = np.arange(0, max_theta + d_theta, d_theta)  # lista de ângulos discretos
        self.vel_range = np.arange(0, max_vel + d_vel, d_vel)  # lista de velocidades discretas

        self.actions = [
            (theta, vel) for theta in self.theta_range for vel in self.vel_range
        ]  # lista de ações de par velocidade-ângulo

    def step(self, action: int, state: tuple):

        theta, vel = self.actions[action]  # utiliza o index da ação selecionada para identificar ângulo e velocidade
        d = 2 * vel ** 2 * np.sin(2 * theta) / 9.81  # calcula ponto final apos o lancamento com parâmetros selecionados

        dist, target_len = state  # distância do alvo
        err = abs(dist - d)  # distância entre o alvo e o ponto atingido (erro)

        reward = -err  # torna o erro uma recompensa negativa

        if err < target_len / 2:  # verifica se o ponto atingido esta dentro da tolerância do alvo
            reward += self.success_reward  # insere recompensa por acerto

        next_state = err  # define o ponto onde o projétil parou como próximo estado

        return reward, next_state

    def reset(self):
        dist = np.random.random() * self.max_dist  # um ponto qualquer entre 0 e a distância máxima
        target_len = np.random.random()*10+4 if self.random_target_len else self.target_len
        return (dist, target_len)


In [ ]:
class Agent:
    # inicia parâmetros do objeto
    def __init__(self, sess: tf.Session, num_actions: int, num_states: int, gamma: float, min_experiences: int, max_experiences: int, batch_size: int):
        self.sess = sess  # sessão do Tensorflow (funcional apenas para TensorFlow v.1)

        self.num_actions = num_actions  # número de possíveis combinacoes para o par angulo-velocidade
        self.min_experiences = min_experiences
        self.max_experiences = max_experiences
        self.batch_size = batch_size

        # placeholders (?)
        self.states_ph = tf.placeholder(tf.float32, shape=(None, num_states))
        self.targets_ph = tf.placeholder(tf.float32, shape=(None,))
        self.actions_ph = tf.placeholder(tf.int32, shape=(None,))

        # experiências
        self.states: list = []
        self.actions: list = []
        self.rewards: list = []
        self.next_states: list = []

        self.gamma = gamma  # taxa de importância de eventos futuros

        fc1 = tf.layers.dense(self.states_ph, 16, activation=tf.nn.relu)  # primeira camada da rede
        fc2 = tf.layers.dense(fc1, 32, activation=tf.nn.relu)  # segunda camada da rede
        self.Q_predicted = tf.layers.dense(fc2, self.num_actions, activation=None)  # camada de saída da rede
        vet_Q_predicted = self.Q_predicted[tf.one_hot(self.actions_ph, self.num_actions, on_value=True, off_value=False)]  # (?)

        loss = tf.losses.mean_squared_error(self.targets_ph, vet_Q_predicted)  # função de perda
        self.optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)  # otimizador

        self.init = tf.global_variables_initializer()  # inicialização das variáveis globais

    # toma uma ação para um dado estado
    def choose_action(self, state: int, eps: float):
        if np.random.random() < eps:
            return np.random.choice(range(self.num_actions))  # toma uma ação aleatória dentre as possibilidades
        return np.argmax(self.predict_one(state))  # toma uma ação através da DQL

    # insere experiências obtidas
    def add_experience(self, state, action, reward, next_state):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.next_states.append(next_state)

    def predict_one(self, state):
        states = np.atleast_2d(state)
        return self.sess.run(self.Q_predicted, feed_dict={self.states_ph: states})

    def train(self):
        if len(self.states) < self.min_experiences:
            return
        
        min_index = m if (m:=len(self.states) - self.max_experiences) >= 0 else 0
        
        if self.batch_size:
            avaliables = range(min_index, len(self.states))
            indexes = np.random.choice(avaliables, size=self.batch_size, replace=False)
        else:
            indexes = range(min_index, len(self.states))
        
        selected_states = [self.states[i] for i in indexes]
        selected_actions = [self.actions[i] for i in indexes]
        selected_rewards = [self.rewards[i] for i in indexes]
        
        targets = selected_rewards  # valor incrementado pelas ações futuras. Não há ações futuras.

        feed_dict = {self.states_ph: selected_states, self.actions_ph: selected_actions, self.targets_ph: targets}
        self.sess.run(self.optimizer, feed_dict=feed_dict)


In [ ]:
param_grid = {
    'theta_disc': np.arange(80, 101, 10),
    'vel_disc': np.arange(90, 111, 10),
    # 'n_episodes': np.arange(1000, 8000, 2000),
    # 'min_experiences': np.arange(0, 1000, 300),
    # 'max_experiences': np.arange(4000, 15000, 3000),
    # 'decay': np.arange(0.001, 0.012, 0.005),
    # 'batch_size': np.arange(0, 1001, 1000),
}

n = 1
for param in param_grid:
    n *= len(param_grid[param])
print('Número de combinações:', n)

In [ ]:
grid = []
start = True
for param in param_grid:
    news = []
    for value in param_grid[param]:
        if start:
            conf = {param: value}
            news.append(conf)
        else:
            for conf in grid:
                new_conf = conf.copy()
                new_conf[param] = value
                news.append(new_conf)
    grid = news
    start = False

In [ ]:
for conf in grid:
    env_settings = {
        'theta_disc': int(conf['theta_disc']),  # número de pontos de discretização do ângulo de lancamento
        'vel_disc': int(conf['vel_disc']),  # número de pontos de dicretização da velocidade de lancamento
        'max_dist': 50,  # máxima distancia possivel para o alvo
        'target_len': 'random'  # comprimento do alvo, isto e, tolerância absoluta para sucesso
    }

    n_states = 2

    n_episodes = 4000  # número de episodios a serem executados
    decay = 0.01  # decaimento da taxa de aleatoriedade

    agent_settings = {
        'num_states': 2,  # número de parâmetros de um estado = (distancia)
        'gamma': 0,  # incremento por ações futuras
        'min_experiences': 300,  # mínimo de experiências aleatórias
        'max_experiences': 10000,  # máximo de experiências aleatórias
        'batch_size': 0  # tamanho do pacote aleatório a ser treinado em cada episódio
    }

    total_reward = 0  # recompensa total
    min_eps = 0.01  # mínima taxa de aleatoriedade
    max_eps = 1  # máxima taxa de aleatoriedade
    verbose = 0  # tipo de output visível após a execução
    pack_size = 5  # números de episódios considerados em cada média no plot resultante

    data = {
        'agent_settings': agent_settings,
        'env_settings': env_settings,
        'training_settings': {
            'n_states': n_states,
            'n_episodes': n_episodes,
            'min_eps': min_eps,
            'max_eps': max_eps,
            'decay': decay
        }
    }

    env = Environment(**env_settings)  # ambiente configurado com parâmetros definidos

    with tf.Session() as sess:
        # saver = tf.train.Saver()
        agent = Agent(sess, len(env.actions), **agent_settings)  # agente configurado com acoes definidas
        sess.run(agent.init)  # inicalização da sessão para o agente (?)

        all_episodes = range(n_episodes)  # index de episódios

        if verbose == 0:
            all_episodes = tqdm(all_episodes)  # barra de progressão por episódios

        start_training = time.time()

        for i in all_episodes:
            eps = min_eps + (max_eps - min_eps) * np.exp(-decay * i)  # cálculo da taxa de aleatoriedade

            state = env.reset()  # gera um novo ambiente = novo alvo
            action = agent.choose_action(state, eps)  # toma uma ação dado o ambiente
            reward, next_state = env.step(action, state)  # calcula os efeitos da ação tomada
            agent.add_experience(state, action, reward, next_state)  # absorve a experiência obtida
            agent.train()  # treino

            if verbose == 1 and i % 100 == 0:
                print("Reward:", reward)

        data['results'] = {
            'training_rewards': agent.rewards,
            'training_time': time.time() - start_training
        }

        print("\nLast 100 episode rewards average:", sum(agent.rewards[-100:]) / 100)
        print("Total reward:", sum(agent.rewards))
        print('Total training time:', data['results']['training_time'], 's')

        # saver.save(sess, 'data/test_session')

        # plot_evolution(agent.rewards, pack_size, 'Treino')

        print('\nTesting.\n')

        n_episodes = 1000

        rewards = []

        for i in tqdm(range(n_episodes)):
            state = env.reset()  # gera um novo ambiente = novo alvo
            action = agent.choose_action(state, 0)  # toma uma ação dado o ambiente
            reward, next_state = env.step(action, state)  # calcula os efeitos da ação tomada
            rewards.append(reward)

        data['results']['testing_rewards'] = rewards
        data['date'] = datetime.today().isoformat()

        if 'data' not in os.listdir():
            os.mkdir('data')
        filename = re.sub(r'[\-\:\.]', '', data['date'])
        with open(f"data/{filename}.json", 'w') as f:
            try:
                json.dump(data, f)
            except Exception as e:
                telegram('Erro no carregamento dos dados.\n\n' + str(e)) # apagar - biblioteca particular
                raise e

        # plot_evolution(rewards, pack_size, 'Teste')

        print('\nScore médio:', sum(rewards)/n_episodes)
telegram('GridSearch completo.') # apagar - biblioteca particular

In [ ]:
data = []
for file in os.listdir('data')[-8:]:
    data.append(json.loads(open(f'data/{file}').read()))

In [ ]:
df = {'vel_disc': [], 'theta_disc': [], 'reward': [], 'training_time': []}
for d in data:
    df['vel_disc'].append(d['env_settings']['vel_disc'])
    df['theta_disc'].append(d['env_settings']['theta_disc'])
    df['reward'].append(np.mean(d['results']['testing_rewards']))
    df['training_time'].append(d['results']['training_time'])
    
result = pd.DataFrame(df)
result

In [ ]:
plt.scatter(result['reward'], result['training_time'])
plt.title('Desempenho das configurações testadas')
plt.xlabel('Recompença média no teste')
plt.ylabel('Tempo de treinamento')
[plt.annotate(i, (result['reward'][i], result['training_time'][i]), xytext=(result['reward'][i]*1.005, result['training_time'][i]*1.005)) for i in result.index]
plt.xlim([min(result['reward'].values)/1.02, max(result['reward'].values)*1.02])
plt.ylim([min(result['training_time'].values)/1.02, max(result['training_time'].values)*1.02])
plt.show()

In [ ]:
result.iloc[3]